In [1]:
# this notebook will use a basic GPT based decision transformer in offline reinforcement learning setting to create bot for trading stock
# get cuda device
# import libraries
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from cust_transf import DecisionTransformer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

# utility function to compute the discounted cumulative sum of a vector
def discount_cumsum(x, gamma):
    disc_cumsum = np.zeros_like(x)
    disc_cumsum[-1] = x[-1]
    for t in reversed(range(x.shape[0]-1)):
        disc_cumsum[t] = x[t] + gamma * disc_cumsum[t+1]
    return disc_cumsum

# utility function to evaluate the performance of the agent on a given environment
# TODO: change it to work with custom environment
def evaluate(model, device, context_len, env, rtg_target, rtg_scale, 
            num_eval_ep=10, max_test_ep_len=28, state_mean=None, state_std=None, render=False):
    eval_batch_size = 1

    results = {}
    total_reward = 0
    total_length = 0

    state_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]

    state_mean = torch.zeros((state_dim,)).to(device) if state_mean is None else torch.from_numpy(state_mean).to(device)
    state_std = torch.ones((state_dim,)).to(device) if state_std is None else torch.from_numpy(state_std).to(device)
    
    # create timestep for transformer
    timesteps = torch.arange(start=0, end=max_test_ep_len, step=1)
    timesteps = timesteps.unsqueeze(0).repeat(eval_batch_size, 1).to(device)

    # evaluate the agent
    model.eval()
    with torch.no_grad():

        for _ in range(num_eval_ep):

            # zeros place holders
            actions = torch.zeros((eval_batch_size, max_test_ep_len, act_dim),
                                dtype=torch.float32, device=device)

            states = torch.zeros((eval_batch_size, max_test_ep_len, state_dim),
                                dtype=torch.float32, device=device)
            
            rewards_to_go = torch.zeros((eval_batch_size, max_test_ep_len, 1),
                                dtype=torch.float32, device=device)
            
            # init episode
            running_state = env.reset()
            running_reward = 0
            running_rtg = rtg_target / rtg_scale

            for t in range(max_test_ep_len):

                total_timesteps += 1

                # add state in placeholder and normalize
                states[0, t] = torch.from_numpy(running_state).to(device)
                states[0, t] = (states[0, t] - state_mean) / state_std

                # calcualate running rtg and add in placeholder
                running_rtg = running_rtg - (running_reward / rtg_scale)
                rewards_to_go[0, t] = running_rtg

                if t < context_len:
                    _, act_preds, _ = model.forward(timesteps[:,:context_len],
                                                states[:,:context_len],
                                                actions[:,:context_len],
                                                rewards_to_go[:,:context_len])
                    act = act_preds[0, t].detach()
                else:
                    _, act_preds, _ = model.forward(timesteps[:,t-context_len+1:t+1],
                                                states[:,t-context_len+1:t+1],
                                                actions[:,t-context_len+1:t+1],
                                                rewards_to_go[:,t-context_len+1:t+1])
                    act = act_preds[0, -1].detach()


                running_state, running_reward, done, _ = env.step(act.cpu().numpy())

                # add action in placeholder
                actions[0, t] = act

                total_reward += running_reward

                if render:
                    env.render()
                if done:
                    break

    results['eval/avg_reward'] = total_reward / num_eval_ep
    results['eval/avg_ep_len'] = total_timesteps / num_eval_ep
    
    return results




In [3]:
from datasets.load import load_dataset
from torch.utils.data import Dataset, DataLoader
# define a custom dataset class which loads the data, modifies the reward to be the discounted cumulative sum and apply trajectory masking
class CustomTrajDataset(Dataset):
    def __init__(self, file_name, context_len, gamma, rtg_scale):
        self.gamma = gamma
        self.context_len = context_len

        # load the data
        data = load_dataset("json", data_files = file_name, field = 'data')
        self.data_state = np.array(data['train']['state'], dtype=np.float32)
        self.data_action = np.array(data['train']['action'], dtype=np.float32)
        self.rtg = np.array(data['train']['reward'], dtype=np.float32)


        # calculate min len, the mean and std of the state and rtg for all data
        self.stateshape = self.data_state.shape
        # calculate mean of state and rtg with numpy
        self.state_mean = np.mean(self.data_state, axis=(-2,-1), keepdims=True)
        self.state_std = np.std(np.abs(self.data_state), axis=(-2,-1), keepdims=True)
        #self.state_mean = torch.mean(data['train']['state'], dim=(-2,-1), keepdim=True)
        #self.state_std = torch.std(data['train']['state'], dim=(-2,-1), keepdim=True)
        self.norm_state = (self.data_state - self.state_mean) / self.state_std

        self.rtg = np.apply_along_axis(discount_cumsum, 1, data['train']['reward'], self.gamma) # type: ignore
        self.rtg = self.rtg / rtg_scale

    def get_state_stats(self):
        return self.state_mean, self.state_std        

    def __len__(self):
        return self.stateshape[0]

    def __getitem__(self, idx):
        state = self.norm_state[idx]
        action = self.data_action[idx]
        rtg = self.rtg[idx]

        data_len = state.shape[0]
        
        if data_len > self.context_len:
            # sample random start index
            start_idx = np.random.randint(0, data_len - self.context_len)
            # slice the data and convert to torch
            state = torch.from_numpy(state[start_idx:start_idx+self.context_len])
            action = torch.from_numpy(action[start_idx:start_idx+self.context_len])
            rtg = torch.from_numpy(rtg[start_idx:start_idx+self.context_len])
            timesteps = torch.arange(start=start_idx, end=start_idx + self.context_len, step=1)
            # trajectory mask
            mask = torch.ones(self.context_len, dtype=torch.long)
        else:
            padding_len = self.context_len - data_len

            # pad the data with zeros
            state = torch.from_numpy(state)
            state = torch.cat([state, torch.zeros((padding_len, *state.shape[1:]))], dim=0)

            action = torch.from_numpy(action)
            action = torch.cat([action, torch.zeros((padding_len, *action.shape[1:]))], dim=0)

            rtg = torch.from_numpy(rtg)
            rtg = torch.cat([rtg, torch.zeros((padding_len, *rtg.shape[1:]))], dim=0)

            timesteps = torch.arange(start=0, end=self.context_len, step=1)

            # trajectory mask
            mask = torch.cat([torch.ones(data_len, dtype=torch.long), torch.zeros(padding_len, dtype=torch.long)], dim=0)
        
        return state, action, rtg, timesteps, mask


In [5]:
# load huggingface dataset from json file
filename = 'AAPL_2190_2016-01-01_1d_random_replaybuffer.json'
context_len = 20
Max_balance = 2147483647

dataset = CustomTrajDataset(filename, context_len = context_len ,gamma = 0.99, rtg_scale = Max_balance)

Using custom data configuration default-3a69897addd28b22
Found cached dataset json (/home/victoru/.cache/huggingface/datasets/json/default-3a69897addd28b22/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 1209.43it/s]


In [6]:
# define training parameters
batch_size = 2
# small learning rate to try to avoid mixed precision caused NaNs
lr = 3e-5
wt_decay = 1e-4
warmup_steps = 10000
n_epochs = 500

In [7]:
# create dataloader from dataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
# define model parameters
# sample 1 batch from dataloader
norm_state, actions, rtg, timestep, traj_mask = next(iter(dataloader))
# use batch shape to determine state dimension
state_dim = norm_state.shape[-1]
act_dim = actions.shape[-1] # discrete action space
# use batch shape to determine context length


n_blocks = 4 # number of transformer blocks
h_dim = 96 # hidden dimension
n_heads = 6 # number of heads in multi-head attention
drop_p = 0.1 # dropout probability


In [9]:
# create the model
model = DecisionTransformer(state_dim, act_dim, n_blocks, h_dim, context_len, n_heads, drop_p).to(device)

# create optimizer
# use larger eps to try to avoid mixed precision overflow caused NaNs
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wt_decay, eps=1e-6)

# create scheduler
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda step: min(1.0, (step + 1) / warmup_steps))

# create a GradScaler for mixed precision training
scaler = torch.cuda.amp.GradScaler(growth_interval=150)
min_scale = 128

In [12]:
# test run the model
with torch.no_grad():
    norm_state, actions, rtg, timestep , traj_mask= next(iter(dataloader))
    norm_state = norm_state.to(device)
    actions = actions.to(device)
    # convert rtg to float
    rtg = rtg.to(device).float()
    timestep = timestep.to(device)
    traj_mask = traj_mask.to(device)
    action_targets = torch.clone(actions).detach().to(device)
    return_preds, state_preds, act_preds = model.forward(norm_state, rtg, timestep, actions)

    # check shape of norm_state
    print(norm_state.shape)
    # check shape of rtg
    print(rtg.shape)
    # check shape of timestep
    print(timestep.shape)
    # check shape of actions
    print(actions.shape)
    print(act_preds.shape)
    print(action_targets.shape)
    
    # consider only the action that are padded
    act_preds = act_preds.view(-1, act_dim)[traj_mask.view(-1) > 0]
    action_targets = action_targets.view(-1, act_dim)[traj_mask.view(-1) > 0]

    # check shape of action targets
    print(action_targets.shape)
    # check shape of action predictions
    print(act_preds.shape)

# check for nan values and inf values in the input and the output of the model
print(torch.isnan(norm_state).any())
print(torch.isnan(rtg).any())
print(torch.isnan(timestep).any())
print(torch.isnan(actions).any())
print(torch.isnan(act_preds).any())
print(torch.isnan(action_targets).any())



torch.Size([2, 20, 13])
torch.Size([2, 20, 1])
torch.Size([2, 20])
torch.Size([2, 20, 2])
torch.Size([2, 20, 2])
torch.Size([2, 20, 2])
torch.Size([40, 2])
torch.Size([40, 2])
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')


In [13]:
print(act_preds)
print(action_targets)
print(norm_state)

tensor([[ 0.5414,  0.4863],
        [ 0.0879,  0.4650],
        [ 0.3458,  0.5598],
        [ 0.0757, -0.3416],
        [-0.0152,  0.6710],
        [ 0.5293,  0.5162],
        [ 0.2007,  0.6124],
        [ 0.3616,  0.5947],
        [ 0.5799,  0.3156],
        [ 0.1769,  0.7526],
        [-0.2917,  0.8149],
        [ 0.1653,  0.3332],
        [ 0.2824,  0.5594],
        [-0.6271,  0.3245],
        [-0.2168,  0.7031],
        [ 0.1997,  0.8121],
        [ 0.6366,  0.5355],
        [ 0.7419,  0.3198],
        [-0.2095,  0.5367],
        [ 0.5437,  0.5196],
        [-0.5138,  0.8217],
        [-0.2318,  0.4292],
        [-0.6535,  0.8390],
        [ 0.0945,  0.4401],
        [ 0.5770,  0.2203],
        [ 0.1478, -0.0228],
        [ 0.2972, -0.0091],
        [-0.2541,  0.3805],
        [ 0.2322,  0.7643],
        [ 0.8399, -0.2268],
        [ 0.2076, -0.4670],
        [ 0.3524,  0.7700],
        [ 0.3907,  0.6256],
        [ 0.8774,  0.4663],
        [ 0.7522,  0.5707],
        [ 0.6727,  0

In [10]:
# create training loop
from tqdm import tqdm

# get the start time to calculate training time
import datetime
start_time = datetime.datetime.now()
#training_log = {'epoch':[], 'loss':[], 'eval/avg_reward':[], 'eval/avg_ep_len':[]} # training log

for i in range(n_epochs):
    model.train()
    log_action_losses = []

    for norm_state, actions, rtg, timestep, traj_mask in tqdm(dataloader):
        # get batch data to device
        norm_state = norm_state.to(device)
        actions = actions.to(device)
        rtg = rtg.to(device).float()
        timestep = timestep.to(device)
        traj_mask = traj_mask.to(device)

        action_targets = torch.clone(actions).detach().to(device)

        # Zeroes out the gradients
        optimizer.zero_grad()

        # run forward pass with autocasting
        # disable autocasting for now to avoid mixed precision caused NaNs
        with torch.cuda.amp.autocast(enabled=False):
            return_preds, state_preds, act_preds = model.forward(norm_state, rtg, timestep, actions)

            # consider only the action that are padded
            act_preds = act_preds.view(-1, act_dim)[traj_mask.view(-1) > 0]
            action_targets = action_targets.view(-1, act_dim)[traj_mask.view(-1) > 0]

            # calculate losses just for actions
            loss = F.mse_loss(act_preds, action_targets, reduction='mean')

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()

        # unscale the gradients
        scaler.unscale_(optimizer)
        # Clips the gradients by norm
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(optimizer)

        # Updates the learning rate according to the scheduler
        scheduler.step()
        # Updates the scale for next iteration.
        scaler.update()
        # enforce min scale to avoid mixed precision caused NaNs
        if scaler.get_scale() < min_scale:
            #print('current scale is ', scaler.get_scale())
            scaler._scale = torch.tensor(min_scale).to(scaler._scale)
            #print('new scale is ', scaler.get_scale())
        
        # append action loss to log
        log_action_losses.append(loss.detach().cpu().item())
    # print every 100 loss log
    if len(log_action_losses) % 10 == 0 or len(log_action_losses) == 1:
        print('Loss: ', log_action_losses[-1])
    

# record training time
end_time = datetime.datetime.now()
print('Training time: ', end_time - start_time)

# create environment to evaluate the model


100%|██████████| 500/500 [00:07<00:00, 69.28it/s]


Loss:  0.44762077927589417


100%|██████████| 500/500 [00:05<00:00, 88.63it/s]


Loss:  0.2813029885292053


100%|██████████| 500/500 [00:05<00:00, 87.62it/s]


...

Loss:  0.19685880839824677


100%|██████████| 500/500 [00:04<00:00, 123.47it/s]


Loss:  0.17367610335350037


100%|██████████| 500/500 [00:04<00:00, 123.59it/s]

Loss:  0.21472211182117462
Training time:  0:34:54.296323
